In [1]:
# Cell 1: Import libraries and load data
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor, DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
import numpy as np

# Initialize Spark session
spark = SparkSession.builder \
    .appName("YieldDataAnalysis") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

# Load the processed dataset
df = spark.read.option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/home/vivi/Downloads/yield_df.csv")

print("Dataset Overview:")
print(f"Number of rows: {df.count()}")
print(f"Number of columns: {len(df.columns)}")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/01 20:45:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

Dataset Overview:
Number of rows: 45706
Number of columns: 21


In [2]:
# Cell 2: Rename columns and correlation analysis
df = df.withColumnRenamed("Yield", "hg/ha_yield") \
       .withColumnRenamed("Rainfall", "average_rain_fall_mm_per_year") \
       .withColumnRenamed("Pesticides", "pesticides_tonnes") \
       .withColumnRenamed("Avg_Temperature", "avg_temp") \
       .withColumnRenamed("Country", "Area") \
       .withColumnRenamed("Crop", "Item")

# Correlation Analysis
print("\n" + "="*60)
print("CORRELATION ANALYSIS")
print("="*60)

numeric_columns = ['hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes', 'avg_temp']

print("Correlation with Yield (hg/ha_yield):")
for col_name in numeric_columns:
    if col_name != 'hg/ha_yield':
        correlation = df.stat.corr('hg/ha_yield', col_name)
        print(f"Correlation between yield and {col_name:35}: {correlation:.4f}")


CORRELATION ANALYSIS
Correlation with Yield (hg/ha_yield):
Correlation between yield and average_rain_fall_mm_per_year      : -0.0279
Correlation between yield and pesticides_tonnes                  : 0.1572
Correlation between yield and avg_temp                           : -0.0584


In [3]:
# Cell 3: Data preparation
print("\n" + "="*60)
print("REGRESSION ANALYSIS - Predicting Crop Yield")
print("="*60)

# Handle categorical variables
area_indexer = StringIndexer(inputCol="Area", outputCol="AreaIndex", handleInvalid="keep")
item_indexer = StringIndexer(inputCol="Item", outputCol="ItemIndex", handleInvalid="keep")

# Use OneHotEncoder for categorical features
area_encoder = OneHotEncoder(inputCol="AreaIndex", outputCol="AreaVec")
item_encoder = OneHotEncoder(inputCol="ItemIndex", outputCol="ItemVec")

# Feature columns
feature_columns = ['AreaVec', 'ItemVec', 'Year', 'average_rain_fall_mm_per_year', 'pesticides_tonnes', 'avg_temp']

# Split the data
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

print(f"Training data count: {train_data.count()}")
print(f"Test data count: {test_data.count()}")

# Initialize evaluators
evaluator_r2 = RegressionEvaluator(labelCol="hg/ha_yield", predictionCol="prediction", metricName="r2")
evaluator_mae = RegressionEvaluator(labelCol="hg/ha_yield", predictionCol="prediction", metricName="mae")
evaluator_rmse = RegressionEvaluator(labelCol="hg/ha_yield", predictionCol="prediction", metricName="rmse")

# Store results for comparison
results = {}


REGRESSION ANALYSIS - Predicting Crop Yield


Training data count: 36698
Test data count: 9008


In [4]:
# Cell 4: Linear Regression
print("\n" + "="*60)
print("1. LINEAR REGRESSION (BASELINE)")
print("="*60)

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
lr = LinearRegression(
    featuresCol="features", 
    labelCol="hg/ha_yield",
    regParam=0.01,
    elasticNetParam=0,
    maxIter=100
)

pipeline_lr = Pipeline(stages=[area_indexer, item_indexer, area_encoder, item_encoder, assembler, lr])
lr_model = pipeline_lr.fit(train_data)
lr_predictions = lr_model.transform(test_data)

lr_r2 = evaluator_r2.evaluate(lr_predictions)
lr_mae = evaluator_mae.evaluate(lr_predictions)
lr_rmse = evaluator_rmse.evaluate(lr_predictions)

print(f"Linear Regression R²: {lr_r2:.4f}")
print(f"Linear Regression MAE: {lr_mae:.2f}")
print(f"Linear Regression RMSE: {lr_rmse:.2f}")

results['Linear Regression'] = {'r2': lr_r2, 'mae': lr_mae, 'rmse': lr_rmse}

# Get coefficients for interpretation
try:
    lr_coefficients = lr_model.stages[-1].coefficients.toArray()
    feature_names = ['Area', 'Item', 'Year', 'Rainfall', 'Pesticides', 'Temperature']
    print("\nLinear Regression Coefficients (interpretation):")
    for i, (name, coef) in enumerate(zip(feature_names, lr_coefficients)):
        impact = "increases" if coef > 0 else "decreases"
        coef_abs = abs(coef)  # Python's abs() not Spark's
        print(f"  {name:15}: {coef:.6f} (each unit {impact} yield by {coef_abs:.2f} hg/ha)")
except Exception as e:
    print(f"Could not extract coefficients: {e}")


1. LINEAR REGRESSION (BASELINE)


25/12/01 20:45:55 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Linear Regression R²: 0.7747
Linear Regression MAE: 22380.59
Linear Regression RMSE: 33423.60

Linear Regression Coefficients (interpretation):
Could not extract coefficients: [NOT_COLUMN_OR_STR] Argument `col` should be a Column or str, got float64.


In [5]:
# Cell 5: Ridge Regression
print("\n" + "="*60)
print("2. RIDGE REGRESSION (L2 REGULARIZATION)")
print("="*60)

ridge = LinearRegression(
    featuresCol="features", 
    labelCol="hg/ha_yield",
    elasticNetParam=0,
    regParam=0.1,
    maxIter=100
)

pipeline_ridge = Pipeline(stages=[area_indexer, item_indexer, area_encoder, item_encoder, assembler, ridge])

try:
    ridge_model = pipeline_ridge.fit(train_data)
    ridge_predictions = ridge_model.transform(test_data)
    
    ridge_r2 = evaluator_r2.evaluate(ridge_predictions)
    ridge_mae = evaluator_mae.evaluate(ridge_predictions)
    ridge_rmse = evaluator_rmse.evaluate(ridge_predictions)
    
    print(f"Ridge Regression R²: {ridge_r2:.4f}")
    print(f"Ridge Regression MAE: {ridge_mae:.2f}")
    print(f"Ridge Regression RMSE: {ridge_rmse:.2f}")
    
    results['Ridge Regression'] = {'r2': ridge_r2, 'mae': ridge_mae, 'rmse': ridge_rmse}
    
except Exception as e:
    print(f"Ridge Regression failed: {e}")


2. RIDGE REGRESSION (L2 REGULARIZATION)
Ridge Regression R²: 0.7747
Ridge Regression MAE: 22380.58
Ridge Regression RMSE: 33423.60


In [6]:
# Cell 6: Lasso Regression
print("\n" + "="*60)
print("3. LASSO REGRESSION (L1 REGULARIZATION)")
print("="*60)

lasso = LinearRegression(
    featuresCol="features", 
    labelCol="hg/ha_yield",
    elasticNetParam=1,
    regParam=0.01,
    maxIter=100
)

pipeline_lasso = Pipeline(stages=[area_indexer, item_indexer, area_encoder, item_encoder, assembler, lasso])

try:
    lasso_model = pipeline_lasso.fit(train_data)
    lasso_predictions = lasso_model.transform(test_data)
    
    lasso_r2 = evaluator_r2.evaluate(lasso_predictions)
    lasso_mae = evaluator_mae.evaluate(lasso_predictions)
    lasso_rmse = evaluator_rmse.evaluate(lasso_predictions)
    
    print(f"Lasso Regression R²: {lasso_r2:.4f}")
    print(f"Lasso Regression MAE: {lasso_mae:.2f}")
    print(f"Lasso Regression RMSE: {lasso_rmse:.2f}")
    
    results['Lasso Regression'] = {'r2': lasso_r2, 'mae': lasso_mae, 'rmse': lasso_rmse}
    
except Exception as e:
    print(f"Lasso Regression failed: {e}")


3. LASSO REGRESSION (L1 REGULARIZATION)
Lasso Regression R²: 0.7747
Lasso Regression MAE: 22380.58
Lasso Regression RMSE: 33423.60


In [7]:
# Cell 7: Elastic Net
print("\n" + "="*60)
print("4. ELASTIC NET REGRESSION")
print("="*60)

elastic_net = LinearRegression(
    featuresCol="features", 
    labelCol="hg/ha_yield",
    elasticNetParam=0.5,
    regParam=0.05,
    maxIter=100
)

pipeline_elastic = Pipeline(stages=[area_indexer, item_indexer, area_encoder, item_encoder, assembler, elastic_net])

try:
    elastic_model = pipeline_elastic.fit(train_data)
    elastic_predictions = elastic_model.transform(test_data)
    
    elastic_r2 = evaluator_r2.evaluate(elastic_predictions)
    elastic_mae = evaluator_mae.evaluate(elastic_predictions)
    elastic_rmse = evaluator_rmse.evaluate(elastic_predictions)
    
    print(f"Elastic Net R²: {elastic_r2:.4f}")
    print(f"Elastic Net MAE: {elastic_mae:.2f}")
    print(f"Elastic Net RMSE: {elastic_rmse:.2f}")
    
    results['Elastic Net'] = {'r2': elastic_r2, 'mae': elastic_mae, 'rmse': elastic_rmse}
    
except Exception as e:
    print(f"Elastic Net failed: {e}")


4. ELASTIC NET REGRESSION
Elastic Net R²: 0.7747
Elastic Net MAE: 22380.47
Elastic Net RMSE: 33423.60


In [8]:
# Cell 8: K-Nearest Neighbors (KNN) 
print("\n" + "="*60)
print("5. K-NEAREST NEIGHBORS REGRESSION (KNN)")
print("="*60)

# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

print("Running KNN analysis...")

# Convert Spark DataFrames to pandas
train_pd = train_data.toPandas()
test_pd = test_data.toPandas()
print(f"Data converted: train={len(train_pd)} rows, test={len(test_pd)} rows")

# Prepare features for KNN
feature_cols_knn = ['Year', 'average_rain_fall_mm_per_year', 'pesticides_tonnes', 'avg_temp']
X_train_knn = train_pd[feature_cols_knn].copy()
X_test_knn = test_pd[feature_cols_knn].copy()

# Add encoded categorical variables
categorical_cols = ['Area', 'Item']
label_encoders = {}

for col in categorical_cols:
    # Fill missing values
    train_pd[col] = train_pd[col].fillna('Unknown')
    test_pd[col] = test_pd[col].fillna('Unknown')
    
    # Create and fit label encoder
    le = LabelEncoder()
    combined = pd.concat([train_pd[col], test_pd[col]], axis=0)
    le.fit(combined)
    
    # Transform the data
    X_train_knn[col] = le.transform(train_pd[col])
    X_test_knn[col] = le.transform(test_pd[col])
    label_encoders[col] = le

# Handle missing values in numerical features
for col in feature_cols_knn:
    if col in X_train_knn.columns:
        X_train_knn[col] = X_train_knn[col].fillna(X_train_knn[col].mean())
        X_test_knn[col] = X_test_knn[col].fillna(X_train_knn[col].mean())

# Scale features (important for KNN)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_knn)
X_test_scaled = scaler.transform(X_test_knn)

# Target variable
y_train = train_pd['hg/ha_yield'].fillna(train_pd['hg/ha_yield'].mean())
y_test = test_pd['hg/ha_yield'].fillna(test_pd['hg/ha_yield'].mean())

# Try different K values
k_values = [3, 5, 7, 10, 15]
knn_results = {}

for k in k_values:
    # Create and train KNN model
    knn = KNeighborsRegressor(n_neighbors=k, weights='distance', n_jobs=-1)
    knn.fit(X_train_scaled, y_train)
    
    # Make predictions
    y_pred = knn.predict(X_test_scaled)
    
    # Calculate metrics
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    # Store results
    knn_results[k] = {'r2': r2, 'mae': mae, 'rmse': rmse}
    
    # Print results for this k
    print(f"KNN with k={k:2d}: R²={r2:.4f}, MAE={mae:.2f}, RMSE={rmse:.2f}")

# Find best K based on highest R² score
best_k = None
best_r2 = -float('inf')
best_mae = None
best_rmse = None

for k, metrics in knn_results.items():
    if metrics['r2'] > best_r2:
        best_r2 = metrics['r2']
        best_k = k
        best_mae = metrics['mae']
        best_rmse = metrics['rmse']

# Print best results
print(f"\nBest KNN: k={best_k}, R²={best_r2:.4f}, MAE={best_mae:.2f}, RMSE={best_rmse:.2f}")

# Store results in results dictionary
results[f'KNN (k={best_k})'] = {'r2': best_r2, 'mae': best_mae, 'rmse': best_rmse}

# Optional: Visualize KNN performance
print("\n" + "="*60)
print("KNN Performance Summary")
print("="*60)
print(f"{'K Value':<10} {'R² Score':<12} {'MAE':<12} {'RMSE':<12}")
print("-" * 60)
for k in sorted(k_values):
    metrics = knn_results[k]
    print(f"{k:<10} {metrics['r2']:<12.4f} {metrics['mae']:<12.2f} {metrics['rmse']:<12.2f}")
print("=" * 60)


5. K-NEAREST NEIGHBORS REGRESSION (KNN)
Running KNN analysis...
Data converted: train=36698 rows, test=9008 rows
KNN with k= 3: R²=0.9682, MAE=4150.71, RMSE=12555.67
KNN with k= 5: R²=0.9540, MAE=5873.35, RMSE=15102.37
KNN with k= 7: R²=0.9405, MAE=7126.25, RMSE=17174.77
KNN with k=10: R²=0.9267, MAE=8442.56, RMSE=19060.57
KNN with k=15: R²=0.9062, MAE=10195.44, RMSE=21560.81

Best KNN: k=3, R²=0.9682, MAE=4150.71, RMSE=12555.67

KNN Performance Summary
K Value    R² Score     MAE          RMSE        
------------------------------------------------------------
3          0.9682       4150.71      12555.67    
5          0.9540       5873.35      15102.37    
7          0.9405       7126.25      17174.77    
10         0.9267       8442.56      19060.57    
15         0.9062       10195.44     21560.81    


In [9]:
# Cell 9: Decision Tree Regressor
print("\n" + "="*60)
print("6. DECISION TREE REGRESSOR")
print("="*60)

distinct_areas = df.select("Area").distinct().count()
distinct_items = df.select("Item").distinct().count()
# Use custom logic instead of max() to avoid conflict with PySpark's max function
max_bins_candidates = [distinct_areas + 1, distinct_items + 1, 64]
max_bins_value = max_bins_candidates[0]
for candidate in max_bins_candidates[1:]:
    if candidate > max_bins_value:
        max_bins_value = candidate

print(f"Distinct areas: {distinct_areas}")
print(f"Distinct crops: {distinct_items}")
print(f"Setting maxBins to: {max_bins_value}")

dt = DecisionTreeRegressor(
    featuresCol="features", 
    labelCol="hg/ha_yield", 
    seed=42, 
    maxDepth=10,
    maxBins=max_bins_value
)

pipeline_dt = Pipeline(stages=[area_indexer, item_indexer, area_encoder, item_encoder, assembler, dt])

try:
    dt_model = pipeline_dt.fit(train_data)
    dt_predictions = dt_model.transform(test_data)
    
    dt_r2 = evaluator_r2.evaluate(dt_predictions)
    dt_mae = evaluator_mae.evaluate(dt_predictions)
    dt_rmse = evaluator_rmse.evaluate(dt_predictions)
    
    print(f"Decision Tree R²: {dt_r2:.4f}")
    print(f"Decision Tree MAE: {dt_mae:.2f}")
    print(f"Decision Tree RMSE: {dt_rmse:.2f}")
    
    results['Decision Tree'] = {'r2': dt_r2, 'mae': dt_mae, 'rmse': dt_rmse}
    
    # Feature Importance
    dt_feature_importances = dt_model.stages[-1].featureImportances
    
    # Create simplified feature importance names
    feature_names = ['Area', 'Item', 'Year', 'Rainfall', 'Pesticides', 'Temperature']
    dt_feature_importance_dict = {}
    for i, name in enumerate(feature_names):
        if i < len(dt_feature_importances):
            dt_feature_importance_dict[name] = dt_feature_importances[i]
        else:
            dt_feature_importance_dict[name] = 0.0
    
    print("\nDecision Tree Feature Importances:")
    dt_items = list(dt_feature_importance_dict.items())
    # Manual bubble sort
    for i in range(len(dt_items)):
        for j in range(i + 1, len(dt_items)):
            if dt_items[i][1] < dt_items[j][1]:
                dt_items[i], dt_items[j] = dt_items[j], dt_items[i]
    
    for feature, importance in dt_items:
        print(f"  {feature:15}: {importance:.4f}")
        
    # Get Decision Tree model details
    dt_model_final = dt_model.stages[-1]
    print(f"\nDecision Tree Details:")
    print(f"  Depth: {dt_model_final.depth}")
    print(f"  Number of nodes: {dt_model_final.numNodes}")
        
except Exception as e:
    print(f"Decision Tree failed: {e}")


6. DECISION TREE REGRESSOR
Distinct areas: 116
Distinct crops: 10
Setting maxBins to: 117
Decision Tree R²: 0.9222
Decision Tree MAE: 11034.37
Decision Tree RMSE: 19643.19

Decision Tree Feature Importances:
  Year           : 0.0317
  Item           : 0.0032
  Pesticides     : 0.0017
  Area           : 0.0003
  Rainfall       : 0.0000
  Temperature    : 0.0000

Decision Tree Details:
  Depth: 10
  Number of nodes: 1221


In [10]:
# Cell 10: Random Forest Regressor
print("\n" + "="*60)
print("7. RANDOM FOREST REGRESSOR")
print("="*60)

rf = RandomForestRegressor(
    featuresCol="features", 
    labelCol="hg/ha_yield", 
    seed=42, 
    maxDepth=10,
    maxBins=max_bins_value,
    numTrees=50
)

pipeline_rf = Pipeline(stages=[area_indexer, item_indexer, area_encoder, item_encoder, assembler, rf])

try:
    rf_model = pipeline_rf.fit(train_data)
    rf_predictions = rf_model.transform(test_data)
    
    rf_r2 = evaluator_r2.evaluate(rf_predictions)
    rf_mae = evaluator_mae.evaluate(rf_predictions)
    rf_rmse = evaluator_rmse.evaluate(rf_predictions)
    
    print(f"Random Forest R²: {rf_r2:.4f}")
    print(f"Random Forest MAE: {rf_mae:.2f}")
    print(f"Random Forest RMSE: {rf_rmse:.2f}")
    
    results['Random Forest'] = {'r2': rf_r2, 'mae': rf_mae, 'rmse': rf_rmse}
    
    # Feature Importance from Random Forest
    rf_feature_importances = rf_model.stages[-1].featureImportances
    rf_feature_importance_dict = dict(zip(feature_names, rf_feature_importances))
    
    print("\nRandom Forest Feature Importances:")
    rf_items = list(rf_feature_importance_dict.items())
    for i in range(len(rf_items)):
        for j in range(i + 1, len(rf_items)):
            if rf_items[i][1] < rf_items[j][1]:
                rf_items[i], rf_items[j] = rf_items[j], rf_items[i]
    
    for feature, importance in rf_items:
        print(f"  {feature:15}: {importance:.4f})")
        
except Exception as e:
    print(f"Random Forest failed: {e}")


7. RANDOM FOREST REGRESSOR


25/12/01 20:46:25 WARN DAGScheduler: Broadcasting large task binary with size 1177.2 KiB
25/12/01 20:46:26 WARN DAGScheduler: Broadcasting large task binary with size 1835.4 KiB
25/12/01 20:46:27 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB
25/12/01 20:46:28 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
                                                                                

Random Forest R²: 0.9176
Random Forest MAE: 12095.49
Random Forest RMSE: 20215.47

Random Forest Feature Importances:
  Year           : 0.0221)
  Item           : 0.0121)
  Area           : 0.0094)
  Rainfall       : 0.0023)
  Pesticides     : 0.0013)
  Temperature    : 0.0013)


In [11]:
# Cell 11: Gradient Boosting Regressor
print("\n" + "="*60)
print("8. GRADIENT BOOSTING REGRESSOR")
print("="*60)

gbt = GBTRegressor(
    featuresCol="features", 
    labelCol="hg/ha_yield", 
    seed=42, 
    maxIter=50,
    maxDepth=6,
    maxBins=max_bins_value
)

pipeline_gbt = Pipeline(stages=[area_indexer, item_indexer, area_encoder, item_encoder, assembler, gbt])

try:
    gbt_model = pipeline_gbt.fit(train_data)
    gbt_predictions = gbt_model.transform(test_data)
    
    gbt_r2 = evaluator_r2.evaluate(gbt_predictions)
    gbt_mae = evaluator_mae.evaluate(gbt_predictions)
    gbt_rmse = evaluator_rmse.evaluate(gbt_predictions)
    
    print(f"Gradient Boosting R²: {gbt_r2:.4f}")
    print(f"Gradient Boosting MAE: {gbt_mae:.2f}")
    print(f"Gradient Boosting RMSE: {gbt_rmse:.2f}")
    
    results['Gradient Boosting'] = {'r2': gbt_r2, 'mae': gbt_mae, 'rmse': gbt_rmse}
    
    # Feature Importance from Gradient Boosting
    gbt_feature_importances = gbt_model.stages[-1].featureImportances
    gbt_feature_importance_dict = dict(zip(feature_names, gbt_feature_importances))
    
    print("\nGradient Boosting Feature Importances:")
    gbt_items = list(gbt_feature_importance_dict.items())
    for i in range(len(gbt_items)):
        for j in range(i + 1, len(gbt_items)):
            if gbt_items[i][1] < gbt_items[j][1]:
                gbt_items[i], gbt_items[j] = gbt_items[j], gbt_items[i]
    
    for feature, importance in gbt_items:
        print(f"  {feature:15}: {importance:.4f})")
    
except Exception as e:
    print(f"Gradient Boosting failed: {e}")


8. GRADIENT BOOSTING REGRESSOR
Gradient Boosting R²: 0.9408
Gradient Boosting MAE: 10113.11
Gradient Boosting RMSE: 17136.24

Gradient Boosting Feature Importances:
  Item           : 0.0175)
  Year           : 0.0157)
  Temperature    : 0.0041)
  Pesticides     : 0.0021)
  Rainfall       : 0.0012)
  Area           : 0.0010)


In [12]:
# Cell 12: Comprehensive Model Comparison
print("\n" + "="*80)
print("COMPREHENSIVE MODEL COMPARISON")
print("="*80)

print(f"{'Model':25} | {'R²':10} | {'MAE':12} | {'RMSE':12} | {'Rank':6}")
print("-" * 80)

# Sort models by R² score
sorted_results = []
for model_name, metrics in results.items():
    sorted_results.append((model_name, metrics['r2'], metrics['mae'], metrics['rmse']))

# Manual sorting (bubble sort)
for i in range(len(sorted_results)):
    for j in range(i + 1, len(sorted_results)):
        if sorted_results[i][1] < sorted_results[j][1]:
            sorted_results[i], sorted_results[j] = sorted_results[j], sorted_results[i]

# Display results with ranking
for rank, (model_name, r2, mae, rmse) in enumerate(sorted_results, 1):
    print(f"{model_name:25} | {r2:10.4f} | {mae:12.2f} | {rmse:12.2f} | {rank:6}")


COMPREHENSIVE MODEL COMPARISON
Model                     | R²         | MAE          | RMSE         | Rank  
--------------------------------------------------------------------------------
KNN (k=3)                 |     0.9682 |      4150.71 |     12555.67 |      1
Gradient Boosting         |     0.9408 |     10113.11 |     17136.24 |      2
Decision Tree             |     0.9222 |     11034.37 |     19643.19 |      3
Random Forest             |     0.9176 |     12095.49 |     20215.47 |      4
Ridge Regression          |     0.7747 |     22380.58 |     33423.60 |      5
Linear Regression         |     0.7747 |     22380.59 |     33423.60 |      6
Elastic Net               |     0.7747 |     22380.47 |     33423.60 |      7
Lasso Regression          |     0.7747 |     22380.58 |     33423.60 |      8


In [13]:
# Cell 13: Performance Interpretation
print("\n" + "="*80)
print("PERFORMANCE INTERPRETATION")
print("="*80)

# Find best model
best_model_name = sorted_results[0][0]
best_r2 = sorted_results[0][1]
best_mae = sorted_results[0][2]
best_rmse = sorted_results[0][3]

# Calculate mean yield for context
mean_yield = df.agg(mean('hg/ha_yield')).collect()[0][0]

if best_r2 > 0.9:
    print(f"EXCEPTIONAL PERFORMANCE - {best_model_name} achieved R² = {best_r2:.4f}")
    print(f"   This means the model explains {best_r2*100:.1f}% of the variance in crop yields")
    print("   This is EXCEPTIONAL predictive power for agricultural data!")
    print("   The model is highly reliable for precision agriculture applications")
elif best_r2 > 0.7:
    print(f"EXCELLENT PERFORMANCE - {best_model_name} achieved R² = {best_r2:.4f}")
    print(f"   This means the model explains {best_r2*100:.1f}% of the variance in crop yields")
    print("   This is EXCELLENT predictive power for agricultural data!")
    print("   The model is highly suitable for yield prediction and forecasting")
elif best_r2 > 0.5:
    print(f"GOOD PERFORMANCE - {best_model_name} achieved R² = {best_r2:.4f}")
    print(f"   This means the model explains {best_r2*100:.1f}% of the variance in crop yields")
    print("   This is GOOD predictive power for agricultural data!")
    print("   The model has moderate predictive capability for yield estimation")
elif best_r2 > 0.3:
    print(f"MODERATE PERFORMANCE - {best_model_name} achieved R² = {best_r2:.4f}")
    print(f"   This means the model explains {best_r2*100:.1f}% of the variance in crop yields")
    print("   This is MODERATE predictive power for agricultural data")
    print("   Consider additional features or data for better performance")
else:
    print(f"POOR PERFORMANCE - {best_model_name} achieved R² = {best_r2:.4f}")
    print(f"   This means the model explains only {best_r2*100:.1f}% of the variance in crop yields")
    print("   This is POOR predictive power for agricultural data")
    print("   Major improvements needed in feature engineering or data collection")

# Additional performance metrics
print(f"\nADDITIONAL PERFORMANCE METRICS:")
print(f"   Mean Absolute Error (MAE): {best_mae:.2f} hg/ha")
print(f"   Root Mean Squared Error (RMSE): {best_rmse:.2f} hg/ha")
print(f"   Mean Yield (baseline): {mean_yield:.2f} hg/ha")
print(f"   MAE relative to mean: {(best_mae/mean_yield*100):.1f}%")
print(f"   RMSE relative to mean: {(best_rmse/mean_yield*100):.1f}%")


PERFORMANCE INTERPRETATION
EXCEPTIONAL PERFORMANCE - KNN (k=3) achieved R² = 0.9682
   This means the model explains 96.8% of the variance in crop yields
   This is EXCEPTIONAL predictive power for agricultural data!
   The model is highly reliable for precision agriculture applications

ADDITIONAL PERFORMANCE METRICS:
   Mean Absolute Error (MAE): 4150.71 hg/ha
   Root Mean Squared Error (RMSE): 12555.67 hg/ha
   Mean Yield (baseline): 75381.22 hg/ha
   MAE relative to mean: 5.5%
   RMSE relative to mean: 16.7%


In [14]:
# Cell 14: Dataset Summary and Cleanup
print("\n" + "="*80)
print("DATASET SUMMARY")
print("="*80)

distinct_areas = df.select("Area").distinct().count()
distinct_items = df.select("Item").distinct().count()
distinct_years = df.select("Year").distinct().count()
min_yield = df.agg(min('hg/ha_yield')).collect()[0][0]
max_yield = df.agg(max('hg/ha_yield')).collect()[0][0]

print(f"• Total records: {df.count():,}")
print(f"• Countries/Areas: {distinct_areas}")
print(f"• Crops/Items: {distinct_items}")
print(f"• Years covered: {distinct_years}")
print(f"• Mean yield: {mean_yield:.2f} hg/ha")
print(f"• Yield range: {min_yield:.0f} - {max_yield:.0f} hg/ha")

# Stop Spark session
spark.stop()
print("\n" + "="*80)
print("ANALYSIS COMPLETE")
print("="*80)


DATASET SUMMARY
• Total records: 45,706
• Countries/Areas: 116
• Crops/Items: 10
• Years covered: 23
• Mean yield: 75381.22 hg/ha
• Yield range: 50 - 347555 hg/ha

ANALYSIS COMPLETE
